# Final Results

Recalculate metrics and plot them, but done at many more stations for each datetime. Basically, a more robust version of [002](./002_calc_some_metrics.ipynb)

In [2]:
import pandas as pd
from pathlib import Path

from typing import TypedDict, Optional

from random import sample
from utils.constants import WINDOW_SIZE, Events, NAN_THRESHOLD, EWM_ALPHA, datetimes
from utils import (
    load_data,
    calc_metrics,
    plot_metrics,
    read_metrics_file,
    plot_metrics_one,
)

import multiprocessing as mp

import warnings


warnings.filterwarnings("ignore")

In [3]:
class StationsToChoose(TypedDict):
    """
    Structure for station selection data.

    Attributes:
        stations: list[str]. Available station names for random sampling.
        num_sample: int. Number of stations to sample from the available list.
    """

    stations: list[str]
    num_sample: int


In [4]:
EVENT: Events = "Forbush Decrease"
MAX_SAMPLES: int = 10  # Samples per date
REPETITION: bool = True  # If True, it will repeat stations already calculated
EWM: bool = True  # If True, it will calculate EWM metrics

event_replace: str = EVENT.replace(" ", "")

## Calculate metrics

In [5]:
def get_valid_stations(
    df: pd.DataFrame, threshold: Optional[float] = None
) -> list[str]:
    """
    Drop stations (columns) that have a high ratio of NaN values.
    The DataFrame is the one with the data of the stations (all.txt).

    Args:
        df (pd.DataFrame): DataFrame with the stations data.
        threshold (Optional[float]): Ratio of NaN values to consider a station invalid.
            If None, it will use the global NAN_THRESHOLD.

    Returns:
        list[str]: List of valid station names (columns).
    """
    if threshold is None:
        threshold = NAN_THRESHOLD

    nans_count = dict(
        filter(
            lambda x: x[1] > 0,
            df.drop(columns="datetime").isna().sum().to_dict().items(),
        )
    )

    total = len(df)

    # Drop columns (stations) that superates a nan ratio threshold
    stations = list(
        df.drop(
            columns=list(
                filter(
                    lambda station: nans_count[station] / total >= threshold, nans_count
                )
            )
        ).columns[1:]
    )

    return stations

In [6]:
stations: dict[str, list[str]] = {
    date: get_valid_stations(
        load_data(f"./data/{event_replace}/{date}/all.txt"),
        threshold=NAN_THRESHOLD,
    )
    for date in datetimes
}

choosen_stations: dict[str, list[str]] = {
    date: list(
        map(
            lambda filename: filename.name.strip().split("_", 1)[0].upper(),
            Path(f"./data/{event_replace}/{date}").glob("*.csv"),
        )
    )
    for date in datetimes
}

stations_to_choose: dict[str, StationsToChoose] = {
    date: {
        # Remove stations already calculated
        # and those that are fixed to be calculated
        "stations": list(
            set(stations[date])
            - set(choosen_stations[date])
            - set(datetimes[date]["stations"].keys())
        ),
        # Final number of samples to choose
        "num_sample": num_samples
        if (
            num_samples := MAX_SAMPLES
            - len(choosen_stations[date])
            - len(datetimes[date]["stations"].keys())
        )
        > 0
        else 0,
    }
    for date in datetimes
}

# Without repetition of stations already calculated
plot_stations = {
    date: sample(items["stations"], k=items["num_sample"])
    + list(datetimes[date]["stations"].keys())
    for date, items in stations_to_choose.items()
}

# Here is added repetition of stations already calculated (if needed)
if REPETITION:
    for date in plot_stations:
        plot_stations[date].extend(choosen_stations[date])

        # Drop duplicates
        plot_stations[date] = list(set(plot_stations[date]))

In [7]:
# 4 minutes approximate to calculate all metrics with my pc
suffix = f"ewm_alpha_{EWM_ALPHA}" if EWM_ALPHA else ""


def prepare_df(path: str) -> pd.DataFrame:
    df = load_data(path).set_index("datetime")
    return df.ewm(alpha=EWM_ALPHA).mean() if EWM_ALPHA and EWM else df


arguments = [
    (
        prepare_df(f"./data/{event_replace}/{date}/all.txt"),
        station,
        date,
        suffix,
    )
    for date, stations in plot_stations.items()
    for station in stations
]

with mp.Pool(processes=mp.cpu_count()) as pool:
    results = pool.starmap(
        calc_metrics,
        arguments,
    )


## Plotting

In [5]:
plot_stations: dict[str, list[str]] = {
    date: list(
        set(
            map(
                # Get Station name from filename
                lambda filename: filename.name.strip().split("_", 1)[0].upper(),
                Path(f"./data/{event_replace}/{date}").glob("*.csv"),
            )
        )
    )
    for date in datetimes
}

In [6]:
# Nice! Expected output
dict(map(lambda x: (x, len(plot_stations[x])), plot_stations))

{'2023-04-23': 10, '2024-03-24': 11, '2024-05-10': 11}

In [7]:
def _mpl_worker_init():
    import os, tempfile  # noqa: E401
    os.environ["MPLCONFIGDIR"] = tempfile.mkdtemp(prefix="mplcache-")

### Two differents plots

In [15]:
def plot_metrics_wrapper(args_tuple: tuple[str, str, int]) -> None:
    date, station, suffix = args_tuple

    df = read_metrics_file(
        event=event_replace,
        date=date,
        station=station,
        window_size=WINDOW_SIZE,
        datetime_cols={"datetime": ""},
        suffix=f"-ewm_alpha_{EWM_ALPHA}" if EWM_ALPHA and EWM else "",
    )

    # Odd suffix: all metrics except "lepel_ziv"
    if suffix % 2 == 1:
        df = df.drop(columns=["lepel_ziv"], errors="ignore")
        relevant_metrics = ["*"]

    else:  # Even suffix: only "lepel_ziv"
        relevant_metrics = ["lepel_ziv"]

    if station in datetimes[date]["stations"] and datetimes[date]["stations"][station]:
        min_datetime, max_datetime = datetimes[date]["stations"][station]
    else:
        min_datetime, max_datetime = datetimes[date]["bounds"]

    plot_metrics(
        window_size=WINDOW_SIZE,
        relevant_metrics=relevant_metrics,
        df=df,
        event=event_replace,
        date=date,
        station=station,
        min_datetime=min_datetime,
        max_datetime=max_datetime,
        freq_hours=2,
        save_format="pdf",
        suffix=str(suffix),
        show=False,
    )


with mp.Pool(processes=mp.cpu_count(), initializer=_mpl_worker_init) as pool:
    arguments_plot = [
        (date, station, suffix + 2 if EWM else suffix)
        for date, stations in plot_stations.items()
        for station in stations
        for suffix in [1, 2]
    ]

    pool.map(plot_metrics_wrapper, arguments_plot)

### One plot

TODO: Change color palette

In [8]:
def plot_metrics_one_wrapper(args_tuple: tuple[str, str]) -> None:
    date, station = args_tuple

    df = read_metrics_file(
        event=event_replace,
        date=date,
        station=station,
        window_size=WINDOW_SIZE,
        datetime_cols={"datetime": ""},
        suffix=f"-ewm_alpha_{EWM_ALPHA}" if EWM_ALPHA and EWM else "",
    )

    if station in datetimes[date]["stations"] and datetimes[date]["stations"][station]:
        min_datetime, max_datetime = datetimes[date]["stations"][station]
    else:
        min_datetime, max_datetime = datetimes[date]["bounds"]

    plot_metrics_one(
        window_size=WINDOW_SIZE,
        relevant_metrics=None,
        df=df,
        event=event_replace,
        date=date,
        station=station,
        min_datetime=min_datetime,
        max_datetime=max_datetime,
        freq_hours=2,
        save_format="pdf",
        show=False,
    )


with mp.Pool(processes=mp.cpu_count(), initializer=_mpl_worker_init) as pool:
    arguments_plot = [
        (date, station)
        for date, stations in plot_stations.items()
        for station in stations
    ]

    pool.map(plot_metrics_one_wrapper, arguments_plot)

RuntimeError: latex was not able to process the following string:
b'Date'

Here is the full command invocation and its output:

latex -interaction=nonstopmode --halt-on-error --output-directory=tmpv9ibwswk 692786374c4f4ec62c35370a0b997bfe9a3e4a3b6d981f90b79aa42ff2bfa672.tex

This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(./692786374c4f4ec62c35370a0b997bfe9a3e4a3b6d981f90b79aa42ff2bfa672.tex
LaTeX2e <2022-11-01> patch level 1
L3 programming layer <2023-02-22>)
! Emergency stop.
<*> ...b997bfe9a3e4a3b6d981f90b79aa42ff2bfa672.tex
                                                  
No pages of output.
Transcript written on tmpv9ibwswk/692786374c4f4ec62c35370a0b997bfe9a3e4a3b6d981
f90b79aa42ff2bfa672.log.


